In [1]:
from PIL import Image

In [2]:
import xarray as xr
from xnoah.xarray import coarsen
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm_notebook as tqdm
import dask.bag as db
from dask.diagnostics import ProgressBar
import os, re



path="/home/disk/eos13/guest/SAM6.10.6_NG/OUT_2D"
file="NG_5120x2560x34_4km_10s_QOBS_EQX_1280_0000858600.2Dcom_001.nc"

! mkdir -p frames


def make_image(args):
    k, file = args
    
    f = xr.open_dataset(file, cache=False)
    i = int(f.time * 84000)
    filename = "frames/%05d.png"%k
    
    if not os.path.exists(filename):
        img = f.CLD.coarsen(x=4, y=4).values
        cmap = plt.get_cmap('bone')
        bits = np.uint8(cmap(img[0,...]/100)*255)
        im = Image.fromarray(bits)
        im.save(filename)
        
    return filename


In [5]:
pattern = "/home/disk/eos13/guest/SAM6.10.6_NG/OUT_2D/NG_5120x2560x34_4km_10s_QOBS_EQX_1280_*.nc"
files = enumerate(sorted(glob(pattern)))

In [6]:
ims = db.from_sequence(files).map(make_image)
ims = ims.compute(num_workers=16)

In [11]:
!ffmpeg  -y -r 30 -f image2 -s 1280x600 -i frames/%05d.png -vcodec libx264 -crf 26  -pix_fmt yuv420p test.mp4

ffmpeg version 2.6.8 Copyright (c) 2000-2016 the FFmpeg developers
  built with gcc 4.8.5 (GCC) 20150623 (Red Hat 4.8.5-4)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -g -pipe -Wall -Wp,-D_FORTIFY_SOURCE=2 -fexceptions -fstack-protector-strong --param=ssp-buffer-size=4 -grecord-gcc-switches -m64 -mtune=generic' --enable-bzlib --disable-crystalhd --enable-gnutls --enable-ladspa --enable-libass --enable-libcdio --enable-libdc1394 --enable-libfaac --enable-nonfree --enable-libfdk-aac --enable-nonfree --disable-indev=jack --enable-libfreetype --enable-libgsm --enable-libmp3lame --enable-openal --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-libschroedinger --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libvorbis --enable-libv4l2 --enable-libx264 --enable-libx265 --enable-libxvid --enable-x11grab --enable-avfilter --enable-a

In [12]:
xr.open_dataset(path+"/"+ file)

<xarray.Dataset>
Dimensions:  (time: 1, x: 5120, y: 2560)
Coordinates:
  * x        (x) float32 0.0 4000.0 8000.0 12000.0 16000.0 20000.0 24000.0 ...
  * y        (y) float32 0.0 4000.0 8000.0 12000.0 16000.0 20000.0 24000.0 ...
  * time     (time) float32 179.875
Data variables:
    Prec     (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    SHF      (time, y, x) float32 -2.36329 -0.863396 -6.05349 -4.18458 ...
    LHF      (time, y, x) float32 23.9857 25.8336 21.9444 27.1687 22.1766 ...
    LWNS     (time, y, x) float32 87.1403 85.2597 77.7035 64.6239 85.2025 ...
    LWNSC    (time, y, x) float32 108.197 108.67 107.153 107.714 107.531 ...
    LWNT     (time, y, x) float32 237.212 244.916 238.165 229.48 239.661 ...
    LWNTC    (time, y, x) float32 250.142 250.103 249.994 250.01 250.19 ...
    SOLIN    (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    SWNS     (time, y, x) float32 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    SWNSC    (time, y, x